# Setup

In [3]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

project root: /Users/joachimpfefferkorn/repos/neurovolume


# Test registration with one frame
Somewhat adapted from a GPT response, let's first make sure that our registration applies to one frame.

In [4]:
bold_image = ants.image_read(bold_stim_filepath)  # Replace with your BOLD image path
t1_image = ants.image_read(t1_input_filepath)     # Replace with your T1 image path

In [6]:
print(len(t1_image.numpy()[0])/2)

256.0


In [17]:
test_slice = ants.from_numpy(bold_image.numpy()[:,:,:,0], spacing=bold_image.spacing[:3])
slice_registration = ants.registration(
    fixed=t1_image,
    moving=test_slice,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
)

bold_slice_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=test_slice,
    transformlist=slice_registration['fwdtransforms'],
    interpolator='linear'
)

In [ ]:
default_cmap = 'nipy_spectral'
def compare_bold_alignment(bold_frame_vol: np.ndarray, anat_vol: np.ndarray, cmap=default_cmap):
    #Just doing the Z dimension for now
    #No time, just a BOLD frame
    # Also just hard coding the slices
    # slice_idx = int(len(anat_vol[0]/2))
    def x_coord(slice_idx):
        fig, axes = plt.subplots(1,3, figsize=(15,5))
        axes[0].imshow(bold_frame_vol[:,:,slice_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,:,slice_idx], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(bold_frame_vol[:,:,slice_idx], cmap='gray')
        axes[2].imshow(anat_vol[:,:,slice_idx], cmap='hot', alpha=0.5)
        axes[2].set_title('Overlay')
    interact(x_coord, slice_idx=(0, anat_vol.shape[2]-1))
    

In [20]:
compare_bold_alignment(bold_slice_registered.numpy(), t1_image.numpy())

interactive(children=(IntSlider(value=147, description='slice_idx', max=295), Output()), _dom_classes=('widget…

In [24]:
print(bold_slice_registered)
print(t1_image)

explore_3D_vol(bold_slice_registered.numpy(), dim='z')
explore_3D_vol(t1_image.numpy(), dim='z')

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=147, description='slice', max=295), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=147, description='slice', max=295), Output()), _dom_classes=('widget-int…

**Looking at this in photoshop, the alignment actually looks a  little off?** Let's try a different transform?

# Previous Failures
It does! but many of the following things we have tried, have not:

## Canonical Way, register all frames within one function

The following does not (it crashes the kernel):

````python
bold_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=bold_image,
    transformlist=registration['fwdtransforms'],
    interpolator='linear',
    imagetype=3
)
````

I suspect this has to do with a glitch on the image type 3 as this was also an issue yesterday. Perhaps open an issue on github

As a workaround, let's use a loop to cycle through all the 4th dimensional stuff.

Given the computational lift of registration, let's write them to a list

## Gathering the Transforms, then registering
The following code proved problematic:
````python
registrations = []
for frame in range(bold_image.shape[3]):
    print(f"frame {frame}/{bold_image.shape[3]}")
    print(" creating bold frame")
    bold_frame = ants.from_numpy(bold_image.numpy()[:,:,:,frame])

    print(" Registering bold frame to T1 image")
    registration = ants.registration(
    fixed=t1_image,
    moving=bold_frame,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
    )
    registrations += registration #This is incorrect! It's just adding the var names
````
When trying to apply these registrations, I got strange behavior. I believe this is because these `registration` values might be getting garbled behind the scenes somewhere. To get around this, let's see if one big function that gathers registrations and applies them does the trick!

Also, perhaps you could have made this a dictionary `img:transforms`

# 4D Registration

In [ ]:
result_np = np.empty((t1_image.shape[0], t1_image.shape[1], t1_image.shape[2], bold_image.shape[3]))

registrations = []
for frame in range(bold_image.shape[3]):
    print(f"frame {frame}/{bold_image.shape[3]}")
    print(" creating bold frame")
    bold_frame = ants.from_numpy(bold_image.numpy()[:,:,:,frame])

    print(" Registering bold frame to T1 image")
    registration = ants.registration(
    fixed=t1_image,
    moving=bold_frame,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
    )
    registrations += registration #This is incorrect! It's just adding the var names